In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

#  Create Networks

In [44]:
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Recurrent neural network
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward Prop
        out, _ = self.rnn(x,h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
    
# Gated Recurrent Unit
class RNN_GRU(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, num_classes):
        super(RNN_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward Prop
        out, _ = self.gru(x,h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out

# Long Short Term Memory
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward Prop
        out, _ = self.lstm(x,(h0, c0))
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)
        return out
    
# Long Short Term Memory only last hidden state
class RNN_LSTM2(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, num_classes):
        super(RNN_LSTM2, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward Prop
        out, _ = self.lstm(x,(h0, c0))
        out = self.fc(out[:,-1,:])
        return out

# verifying the network is working

# set device

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# hyperparameters

In [39]:
input_size = 28
squence_length = 28
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 2

# load Data

In [40]:
train_dataset = datasets.MNIST(root = 'dataset/', train = True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


# Initialize network

In [45]:
model = RNN(input_size, hidden_size, squence_length, num_layers, num_classes).to(device)
model_gru = RNN_GRU(input_size, hidden_size, squence_length, num_layers, num_classes).to(device)
model_lstm = RNN_LSTM(input_size, hidden_size, squence_length, num_layers, num_classes).to(device)
model_lstm2 = RNN_LSTM2(input_size, hidden_size, squence_length, num_layers, num_classes).to(device)

# Loss and optimizer

In [57]:
criterion = nn.CrossEntropyLoss()
criterion_g = nn.CrossEntropyLoss()
criterion_l = nn.CrossEntropyLoss()
criterion_l2 = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr = learning_rate)
optimizer_g = optim.Adam(model_gru.parameters(), lr = learning_rate)
optimizer_l = optim.Adam(model_lstm.parameters(), lr = learning_rate)
optimizer_l2 = optim.Adam(model_lstm2.parameters(), lr = learning_rate)

# Train Network

In [58]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        #Gat data to cuda if possible
        data = data.to(device = device).squeeze(1)
        targets = targets.to(device = device)
        
        
        # forward for RNN
        scores = model(data)
        loss = criterion(scores, targets)
        
        # backward prop
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent or adam step
        optimizer.step()
        
        
        # forward for GRU
        scores_g = model_gru(data)
        loss_g = criterion_g(scores_g, targets)
        
        # backward prop
        optimizer_g.zero_grad()
        loss_g.backward()
        
        # gradient descent or adam step
        optimizer_g.step()
        
        
        # forward for LSTM
        scores_l = model_lstm(data)
        loss_l = criterion_l(scores_l, targets)
        
        # backward prop
        optimizer_l.zero_grad()
        loss_l.backward()
        
        # gradient descent or adam step
        optimizer_l.step()
        
        
        # forward for LSTM only last hidden layer
        scores_l2 = model_lstm2(data)
        loss_l2 = criterion_l2(scores_l2, targets)
        
        # backward prop
        optimizer_l2.zero_grad()
        loss_l2.backward()
        
        # gradient descent or adam step
        optimizer_l2.step()

# Check accuracy on training & test to see how good our model

In [54]:
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device).squeeze(1)
            y = y.to(device = device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
        print(f' Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    model.train()
    

In [59]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

check_accuracy(train_loader, model_gru)
check_accuracy(test_loader, model_gru)

check_accuracy(train_loader, model_lstm)
check_accuracy(test_loader, model_lstm)

check_accuracy(train_loader, model_lstm2)
check_accuracy(test_loader, model_lstm2)

 Got 55886 / 60000 with accuracy 93.14
 Got 9224 / 10000 with accuracy 92.24
 Got 58874 / 60000 with accuracy 98.12
 Got 9781 / 10000 with accuracy 97.81
 Got 59159 / 60000 with accuracy 98.60
 Got 9845 / 10000 with accuracy 98.45
 Got 58958 / 60000 with accuracy 98.26
 Got 9810 / 10000 with accuracy 98.10
